# Making The SVM$_\Delta$+ Fair
[This paper](https://arxiv.org/pdf/1507.05259v3.pdf) by Zafar et al. sets out a cool idea. Let's learn a fair classifier. 


The idea is to add constraints, so 

$$
\begin{aligned}
\textrm{minimize} \quad & L( \theta ) \\ 
\textrm{subject to} \quad & \frac{1}{N}\sum_{i=1}^{N}(z_i - \bar{z})d_\theta (x_i) \leq c \\
\textrm{and} \quad & \frac{1}{N}\sum_{i=1}^{N}(z_i - \bar{z})d_\theta (x_i) \geq -c \\
\end{aligned}
$$

Where $L$ is the loss function on some weights $\theta$, $c$ is the covariance threshold, specifying an upper-bound on the covariance between each sensitive attribute ($z_i$) and the **signed** distance from the feature vector ($x_i$) to the decision boundary ($d_\theta (x_i)$). For clarity, $\bar{z}$ is the mean average position of the sensitive features $z$.

This is meant to be a general form for classifiers. But we're using an SVM, so here's my argument. If we have the unsigned distance, $y_id_\theta (x_i)$ couldn't we just have one constraint? So $\frac{1}{N}\sum_{i=1}^{N}(z_i - \bar{z})y_i d_\theta (x_i) \leq c$?


No. We can't because using the $y_i$ value to make the value unsigned is all very well and good, until you have something that just doesn't classify correctly. We still want those to be with the covariance limit.

The other option could be to incorporate the slack variable $\xi_i$ into the constraint, so $\frac{1}{N}\sum_{i=1}^{N}(z_i - \bar{z})(y_i d_\theta (x_i) + \xi_i)\leq c$. I *think* the reason this wouldn't have been suggested is that the slack variable $\xi$ will always ensure that the margin is ensured. This doesn't really help if we want to consider the true distance to the decision boundary.

What we have to do is keep the two fairness constraints and incorporate them into the SVM$_\Delta$+. Let's see what that looks like.

$$
\begin{aligned}
\min_{w, w*, b, b*} \quad & \frac{1}{2}[(w,w)+\gamma(w^*,w^*)] + C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+\zeta_i]+\Delta C \sum_{i=1}^{\ell}\zeta_i \\ 
\textrm{subject to} \quad & y_i((w,x_i)+b) \geq 1 - y_i ((w^*,x_i^*) + b^*) - \zeta_i \\
\textrm{and} \quad & y_i((w^*,x_i^*)+b^*)+\zeta_i \geq 0 \\
\textrm{and} \quad & \zeta \geq 0 \\
\textrm{and} \quad & \frac{1}{\ell}\sum_{i=1}^{\ell}(z_i - \bar{z})((w,x)+b) (x_i) \leq v \\
\textrm{and} \quad & \frac{1}{\ell}\sum_{i=1}^{\ell}(z_i - \bar{z})((w,x)+b) \geq -v \\
\end{aligned}
$$

Now this actaully looks quite nice. But here's the problem. If you follow this form through, st one point we will want to differnentiate with respect to $w$ and we end up with $x_i(z_i-\bar{z})$ which doesn't work as by definition $z_i$ is a subset of $x_i$, so they almost certainly won't have the same dimensions. Because of this we're going to keep the distance to the decision boundary in the form $d_\theta(x_i)$. Though for the sake of simplicity, we're just going to call it $d(x_i)$.

### Nomenclature

- $w$
    - The weight vector we want to learn in $X$ space
    
- $w*$
    - The weight vector we want to learn in $X^*$ space
    
- $\gamma$
    - Hyperparameter for controlling the affect of $X^*$ space on $X$ space
    
- $C$
    - Hyperparameter for controlling the affect of the slack variables across both spaces on the decision boundary in $X$ space

- $\zeta$
    - Describes the relationship between the slacks in $X$ and $X^*$ space. $\zeta = \xi - \xi^*$ where $\zeta \geq 0$

- $\Delta$
    - This one is a bit up the air tbh. It's described as $\Delta \geq 0$ and being "sufficiently large". My understanding is that the $C$ hyperparemeter controls the slacks in both spaces simultaneously, but the $\Delta$ hyperparameter gives a way of controlling the slack in just the $X$ space. **THIS MY BE WRONG THOUGH**

- $b$
    - The bias we are trying to learn in $X$ space

- $b^*$
    - The bias we are trying to learn in $X^*$ space
    
- $z$
    - The *sensitive* feature(s) we want to ensure fairness over.
    
- $\bar{z}$
    - The average vector of the sensitive feature(s).
    
- $v$
    - The covariance threshold. This is a hyperparameter that trades off fairness for accuracy.
    
- $d(x_i)$
    - Distance to the decision boundary. $d(x_i) = (w,x_i)+b$

## Converting this Primal problem to it's Dual

### Step 1 - Get everything in the right form

we need the problem in the form 
$$
\begin{aligned}
\min_{s} \quad & f(s) \\
\textrm{subject to} \quad & g(s) \leq 0
\end{aligned}
$$

Giving us 

$$
\begin{aligned}
\min_{w, w*, b, b* , \zeta} \quad & \frac{1}{2}[(w,w)+\gamma(w^*,w^*)] + C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+\zeta_i]+\Delta C \sum_{i=1}^{\ell}\zeta_i \\ 
\textrm{subject to} \quad & 1 - y_i ((w^*,x_i^*) + b^*) - \zeta_i - y_i((w,x_i)+b) \leq 0 \\
\textrm{and} \quad & -y_i((w^*,x_i^*)+b^*)-\zeta_i \leq 0 \\
\textrm{and} \quad & -\zeta \leq 0 \\
\textrm{and} \quad & \frac{1}{\ell}[\sum_{i=1}^{\ell}(z_i - \bar{z})d(x_i)] - v \leq 0 \\
\textrm{and} \quad & -\frac{1}{\ell}[\sum_{i=1}^{\ell}(z_i - \bar{z})d(x_i)] -v \leq 0 \\
\end{aligned}
$$

Though if we follow what Vapnik gives us the last term of the objective quickly gets changed to

$$
\begin{aligned}
\min_{w, w*, b, b* , \zeta} \quad & \frac{1}{2}[(w,w)+\gamma(w^*,w^*)] + C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+ (1 + \Delta ) \zeta_i ] \\ 
\textrm{subject to} \quad & 1 - y_i ((w^*,x_i^*) + b^*) - \zeta_i - y_i((w,x_i)+b) \leq 0 \\
\textrm{and} \quad & -y_i((w^*,x_i^*)+b^*)-\zeta_i \leq 0 \\
\textrm{and} \quad & -\zeta \leq 0 \\
\textrm{and} \quad & \frac{1}{\ell}[\sum_{i=1}^{\ell}(z_i - \bar{z})d(x_i)] - v \leq 0 \\
\textrm{and} \quad & -\frac{1}{\ell}[\sum_{i=1}^{\ell}(z_i - \bar{z})d(x_i)] -v \leq 0 \\
\end{aligned}
$$

### Step 2 - Convert to Lagrangian form

which, just to remind ourselves, is... 
$$
L(s,\alpha) = f(s) + \alpha g(s)
$$

Giving us the problem

$$
L(w , w^* , b , b^* , \zeta , \alpha , \beta , \Gamma , \lambda , \mu) = \frac{1}{2}[(w,w)+\gamma(w^*,w^*)] + C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+(1+\Delta )\zeta_i ] + \sum_{i=1}^{\ell}\alpha_i[1 - y_i ((w^*,x_i^*) + b^*) - \zeta_i - y_i((w,x_i)+b)] + \sum_{i=1}^{\ell}\beta_i[-y_i((w^*,x_i^*)+b^*)-\zeta_i] + \sum_{i=1}^{\ell}\Gamma_i[-\zeta_i] + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i - \bar{z})d(x_i)] - v] + \sum_{i=1}^{\ell}\mu_i[-\frac{1}{\ell}[(z_i - \bar{z})d(x_i)] -v]
$$

For the sake of simplicity, let's neaten that up a tiny bit

$$
L(w , w^* , b , b^* , \zeta , \alpha , \beta , \Gamma , \lambda , \mu) = \frac{1}{2}[(w,w)+\gamma(w^*,w^*)] + C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+(1+\Delta )\zeta_i ] + \sum_{i=1}^{\ell}\alpha_i[1 - y_i ((w^*,x_i^*) + b^*) - \zeta_i - y_i((w,x_i)+b)] - \sum_{i=1}^{\ell}\beta_i[y_i((w^*,x_i^*)+b^*)+\zeta_i] - \sum_{i=1}^{\ell}\Gamma_i[\zeta_i] + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i - \bar{z})d(x_i)] - v] - \sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i - \bar{z})d(x_i)]+v]
$$

### Step 3 - Find the 0 vector for this equation
In other words, differentiate over each variable  we want to minimize to find where the gradient = 0

$$
\begin{aligned}
\frac{\partial L}{\partial w} & = w - \sum_{i=1}^{\ell}\alpha_i y_i x_i = 0 \\
& \equiv w - \sum_{i=1}^{\ell}\alpha_i y_i x_i = 0 \\
& \equiv w = \sum_{i=1}^{\ell}\alpha_i y_i x_i \\
\frac{\partial L}{\partial w^*} & = \gamma w^* + \sum_{i=1}^{\ell} C y_i x^*_i - \alpha_i y_i x^*_i - \beta_i y_i x^*_i = 0 \\
& \equiv \gamma w^* + \sum_{i=1}^{\ell} y_i x^*_i (-\alpha - \beta + C) = 0 \\
& \equiv w^* = -\frac{1}{\gamma}\sum_{i=1}^{\ell} y_i x^*_i (-\alpha - \beta + C) \\
& \equiv w^* = \frac{1}{\gamma}\sum_{i=1}^{\ell} y_i x^*_i (\alpha + \beta - C) \\
\frac{\partial L}{\partial b} & = \sum_{i=1}^{\ell} -\alpha_i y_i = 0 \\
& \equiv \sum_{i=1}^{\ell} \alpha_i y_i = 0 \\
\frac{\partial L}{\partial b^*} & = \sum_{i=1}^{\ell}Cy_i - \alpha_iy_i - \beta_iy_i = 0 \\
& \equiv \sum_{i=1}^{\ell}y_i(-\alpha -\beta+C) = 0 \\
\frac{\partial L}{\partial \zeta} & = \sum_{i=1}^{\ell}C(1+\Delta)-\alpha_i-\beta_i-\Gamma_i = 0 \\
& \equiv \sum_{i=1}^{\ell}(C + \Delta C) -\alpha_i-\beta_i-\Gamma_i = 0 \\
& \equiv \sum_{i=1}^{\ell}(C + \Delta C) = \sum_{i=1}^{\ell}\alpha_i+\beta_i+\Gamma_i \\
\text{in Vapnik this is } & (C + \Delta C) = \alpha_i + \beta_i +\Gamma_i \\
& \text{not convinced myself of the leap yet}
\end{aligned}
$$

### Step 4 - Put these back into the Lagrangian

Bu that thing's quite big, so let's do it a term at a time

#### 1st Term

$$
\frac{1}{2}[(w,w)+\gamma(w^*,w^*)] \\
\equiv \frac{1}{2}[\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\gamma(\frac{1}{\gamma}\frac{1}{\gamma}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C))] \\
\equiv \frac{1}{2}[\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+(\frac{1}{\gamma}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C))] \\
\equiv \frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) \\
$$

#### 2nd Term

$$
C \sum_{i=1}^{\ell}[y_i((w^*,x_i^*)+b^*)+(1+\Delta )\zeta_i ] \\
\equiv \sum_{i=1}^{\ell} C[y_i((w^*,x_i^*)+b^*) + C((1+\Delta )\zeta_i] \\
\equiv \sum_{i=1}^{\ell} C[y_i((w^*,x_i^*)+b^*) + (C+\Delta C)\zeta_i] \\
\equiv \sum_{i=1}^{\ell} [Cy_i(\frac{1}{\gamma}y_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+b^*) + (C+\Delta C)\zeta_i] \\
\equiv \sum_{i=1}^{\ell} [Cy_i(\frac{1}{\gamma}y_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+b^*) + \alpha_i\zeta_i+\beta_i\zeta_i+\Gamma\zeta_i] \\
\equiv \sum_{i=1}^{\ell} [\frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+Cy_ib^* + \alpha_i\zeta_i+\beta_i\zeta_i+\Gamma\zeta_i]
$$

This is far as I feel I can take this one for now. Let's move onto the others.

#### 3rd Term


$$
\sum_{i=1}^{\ell}\alpha_i[1-y_i((w^*,x^*)+b^*) - \zeta_i - y_i((w,x)+b)] \\
\equiv \sum_{i=1}^{\ell} \alpha_i -\alpha_iy_i((w^*,x^*)+b^*) - \alpha_i\zeta_i - \alpha_iy_i((w,x),+b) \\
\equiv \sum_{i=1}^{\ell} \alpha_i - \alpha_i\zeta_i - \alpha_iy_i(\frac{1}{\gamma}y_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C)+b^*) -\alpha_iy_i((\alpha_jy_j(x_i,x_j))+b)\\
\equiv \sum_{i=1}^{\ell} \alpha_i - \alpha_i\zeta_i - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) - \alpha_iy_ib^* -\alpha_i\alpha_jy_iy_j(x_i,x_j)-\alpha_iy_ib\\
$$

#### 4th Term

$$
-\sum_{i=1}^{\ell}\beta_i[y_i((w^*,x_i^*)+b^*)+\zeta_i] \\
\equiv -\sum_{i=1}^{\ell}\beta_iy_i((w^*,x_i^*)+b^*)+\beta_i\zeta_i] \\
\equiv -\sum_{i=1}^{\ell}\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C)+\beta_iy_ib^* +\beta_i\zeta_i \\
\equiv \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C)-\beta_iy_ib^* -\beta_i\zeta_i \\
$$

#### 5th Term

$$
-\sum_{i=1}^{\ell}\Gamma_i\zeta_i
$$

#### 6th Term

$$
\sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] 
$$

#### 7th Term

$$
-\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

#### All together

$$
\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} [\frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+Cy_ib^* + \alpha_i\zeta_i+\beta_i\zeta_i+\Gamma\zeta_i] + \sum_{i=1}^{\ell} \alpha_i - \alpha_i\zeta_i - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) - \alpha_iy_ib^* -\alpha_i\alpha_jy_iy_j(x_i,x_j)-\alpha_iy_ib + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C)-\beta_iy_ib^* -\beta_i\zeta_i -\sum_{i=1}^{\ell}\Gamma_i\zeta_i + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv \frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+Cy_ib^* + \sum_{i=1}^{\ell} \alpha_i - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) - \alpha_iy_ib^* -\alpha_i\alpha_jy_iy_j(x_i,x_j)-\alpha_iy_ib + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C)-\beta_iy_ib^* + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)+Cy_ib^* + \sum_{i=1}^{\ell} \alpha_i - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) - \alpha_iy_ib^* -\alpha_iy_ib + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C)-\beta_iy_ib^* + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C) + \sum_{i=1}^{\ell} \alpha_i - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v]  -\alpha_iy_ib - \alpha_iy_ib^* -\beta_iy_ib^* +Cy_ib^*
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C) - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v]  -\alpha_iy_ib + b^*y_i(-\alpha_i -\beta_i +C)
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C) - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C) + \sum_{i=1}^{\ell}-\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] - \frac{1}{\gamma}\alpha_iy_iy_j(x^*_i,x^*_j)(\alpha_j+\beta_j-C)+ \sum_{i=1}^{\ell} \frac{1}{\gamma}Cy_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C) - \sum_{i=1}^{\ell}\frac{1}{\gamma}\beta_iy_iy_j(x_i^*,x^*_j)(\alpha_j+\beta_j-C) 
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)+\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] - \sum_{i=1}^{\ell}\frac{1}{\gamma}y_iy_j(x_i^*,x_j^*)(\alpha_i+\beta_i-C)(\alpha_j+\beta_j-C)
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)-\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})d(x_i)]+v] 
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)-\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[(z_i-\bar{z})\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[(z_i-\bar{z})\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)]+v]
$$

$$
\equiv \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)-\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]+v]
$$

## I'm a bit lost now tbh.... Where were we..

### Oh, we want to maximise

Over our Lagrangian variables. For the sake of simplicity we say $\delta = C- \beta$ in $(\alpha-\delta)$

$$
\max_{\alpha , \delta , \lambda , \mu} \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)-\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) + \sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]+v]
$$

Subject to the constraints

$$
\sum_{i=1}^{\ell}y_i\alpha_i = 0 \\
\sum_{i=1}^{\ell}y_i\delta_i = 0 \\
0 \leq \alpha \leq \delta_i +\Delta C \\
0 \leq \delta \leq C \\
\lambda \geq 0 \\
\mu \geq 0
$$

You might wonder how some of the constraints have upper bounds. It's all from rearranging the equations we got from differentiating. I've done them with pen and paper and they're not hard. But would be distracting to have them here now tbh. The big one is remember $\Gamma \geq 0$.

## This is where I'm stuck and I dont know how to progress

I feel **fairly** confident about the above. It looks how I expected it to anyway and it's been a useful exercise in deriving the SVM$_\Delta$+. 

I say it looks like I expected it to as there's the SVM$_\Delta$+ with the fairness constraints in the main optimization problem. Here's the thing... I can't get it in the right form. I need to separate $\sum_{i=1}^{\ell}\lambda_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]-v] -\sum_{i=1}^{\ell}\mu_i[\frac{1}{\ell}[\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})]+v]$ into the form 
$\begin{pmatrix} exp1 & exp2 & exp3 & exp4 \end{pmatrix}\times\begin{pmatrix}\alpha \\ \delta \\ \lambda \\ \mu \end{pmatrix}$
and I've no idea how to do that. I've got to $\alpha+\alpha\frac{1}{\ell}(yxx(\lambda z_i - \mu z_i - \lambda\bar{z} + \mu\bar{z}))+\lambda v -\mu v$ which feels tantalisingly close, I just don't know how to break up the $\alpha$'s, $\lambda$'s and $\mu$'s.

The only thing I can think to do is break them out of the lagrangian and add them again to the constraints. I'm not sure if that's allowed or not (but it seems to work).

$$
\begin{aligned}
\max_{\alpha , \delta} & \sum_{i=1}^{\ell} \alpha_i -\frac{1}{2}\sum_{i=1}^{\ell}\alpha_i\alpha_jy_iy_j(x_i,x_i)-\frac{1}{2\gamma}\sum_{i,j=1}^{\ell}y_iy_j(x^*_i, x^*_j)(\alpha_i + \beta_i -C)(\alpha_j + \beta_j - C) \\
\textrm{subject to} \quad & \sum_{i=1}^{\ell}y_i\alpha_i = 0 \\
\textrm{and} \quad & \sum_{i=1}^{\ell}y_i\delta_i = 0 \\
\textrm{and} \quad & 0 \leq \alpha \leq \delta_i +\Delta C \\
\textrm{and} \quad & 0 \leq \delta \leq C \\
\textrm{and} \quad & \sum_{i=1}^{\ell}[\frac{1}{\ell}\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})] \leq v \\
\textrm{and} \quad & -\sum_{i=1}^{\ell}[\frac{1}{\ell}\sum_{j=1}^{\ell}\alpha_jy_j(x_i,x_j)(z_i-\bar{z})] \leq v
\end{aligned}
$$
